<span style="font-family:Papyrus; font-size:3em;">Homework 2</span>

<span style="font-family:Papyrus; font-size:2em;">Cross Validation</span>

# Problem

In this homework, you will analyze the effect on the quality of the model fit and parameter estimates
as a result of the number of parameters and the noise in the observational data.
For a two parameter model, use $S_1 \xrightarrow{v_1} S_2 \xrightarrow{v_3} S_3$,
where $v_i = k_i s_i$, $k_i$ is a parameter to estimate, and $s_i$ is the concentration of $S_i$.
The initial concentration of $S_1 = 10$, and the true value of $k_i$ is $i$.
You will design experiments, implement codes to run them, run the experiments, and interpret the results.
You will produce two tables structure as the one below, one with $R^2$ values for the factor levels
and the other with the average parameter std.

 |        | std = 0.5 | std=1.0 | std=1.5
 ------   | --------- | ------- | -------
 numParameter=2 | ? | ? | ?
 numParameter=4 | ? | ? | ?
 numParameter=6 | ? | ? | ?
 

1. Write the 3 models in Antimony, and produce plots for their true values. Use a simulation time
of 10 and 100 points.
1. Write a function that creates synthetic data given the parameters std and numParameter.
1. Extend the AlignFitter class in ``common/util_crossvalidation.py`` by adding the method
``calcAvgParameterStd``. You will do this by creating a new class, ``ExtendedFitter`` that
subclasses ``AligningFitter``.
1. Run experiments that produce the tables described above.
1. Using the baseline std=1.0, numParameter=4, calculate $\mu$, $\alpha_i,k_i$,
$\gamma_i,i_k,j,k_j$.
1. Answer the following questions
   1. Are there any significant interaction effects?
   1. What is the general effect on $R^2$ and average parameter std as the number of parameters increases?
   1. What does the noise standard deviation influence quality, especially as the number of parameters increases?

# Programming Preliminaries
This section provides the setup to run your python codes.

In [5]:
IS_COLAB = False
#
if IS_COLAB:
  !pip install tellurium
  !pip install SBstoat
#    
# Constants for standalone notebook
if not IS_COLAB:
    CODE_DIR = "/home/ubuntu/advancing-biomedical-models/common"
else:
    from google.colab import drive
    drive.mount('/content/drive')
    CODE_DIR = "/content/drive/My Drive/Winter 2021/common"
import sys
sys.path.insert(0, CODE_DIR)

In [6]:
CODE_DIR

'/home/ubuntu/advancing-biomedical-models/common'

In [7]:
import util_crossvalidation as ucv

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tellurium as te